In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import glob
from PIL import Image
from random import shuffle

In [2]:
train_files = glob.glob("C:/Users/anwes/Downloads/archive (2)/flowers/*/**")   #now has all image files
shuffle(train_files)

In [3]:
def find_flower_name(fl):
    if(fl.find("astilbe")!= -1 ):
        return 1
    if(fl.find("bellflower")!= -1 ):
        return 2
    if(fl.find("black_eyed_susan")!= -1 ):
        return 3
    if(fl.find("calendula")!= -1 ):
        return 4
    if(fl.find("california_poppy")!= -1 ):
        return 5
    if(fl.find("carnation")!= -1 ):
        return 6
    if(fl.find("common_daisy")!= -1 ):
        return 7
    if(fl.find("coreopsis")!= -1 ):
        return 8
    if(fl.find("daffodil")!=-1):
        return 9
    if(fl.find("dandelion")!=-1):
        return 10
    if(fl.find("iris")!=-1):
        return 11
    if(fl.find("magnolia")!=-1):
        return 12
    if(fl.find("rose")!=-1):
        return 13
    if(fl.find("sunflower")!=-1):
        return 14
    if(fl.find("tulip")!=-1):
        return 15
    if(fl.find("water_lily")!=-1):
        return 16

In [4]:
X_train=np.zeros((len(train_files),28,28,1))
Y_train=np.zeros((len(train_files),1))

for i, file in enumerate(train_files):
    #i stores index
    #file stores image path
    
    img = Image.open(file)
    
    #resizing the image from original size to (128,128,3) so that it fits X_train array
    img = img.resize((28,28))

    #converting all RGB to grayscale #now image array dimension is (128,128)
    img = img.convert('L')
#     plt.imshow(img)
    
    
    img_array=np.array(img) 
    img_array = img_array.reshape((28,28,1))
#     print(img_array.shape ) 
    
    if( img_array.shape == (28,28,1)): #checking condition so only valid dimension of image array is entered into X_train
        X_train[i,:,:,:]=img_array
# # #     print(X_train)
    
    Y_train[i] = find_flower_name(train_files[i])

In [5]:
X_train[0].shape

(28, 28, 1)

In [6]:
Y_train.shape

(15740, 1)

In [7]:
#two types selectors functions: 1. INT  2. Choice
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),#min value of filter you want
        kernel_size=hp.Choice('conv_1_kernel', values = [2,4]),#picks like mcq
        activation='relu',
        input_shape=(28,28,1)
    ),
      #for i in range(0,hp.Int(....)) this will also work
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),# ihave added this line extra in comparison to the old model
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(40, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [8]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [9]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist34569Fashion")

INFO:tensorflow:Reloading Oracle from existing project output\Mnist34569Fashion\oracle.json
INFO:tensorflow:Reloading Tuner from output\Mnist34569Fashion\tuner0.json


In [10]:
tuner_search.search(X_train,Y_train,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 48s]
val_accuracy: 0.07369758933782578

Best val_accuracy So Far: 0.20775094628334045
Total elapsed time: 00h 02m 37s
INFO:tensorflow:Oracle triggered exit


In [11]:
model=tuner_search.get_best_models(num_models=1)[0]

In [12]:
model.fit(X_train,Y_train, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
443/443 [==============================] - 17s 38ms/step - loss: 1.8144 - accuracy: 0.4403 - val_loss: 1.9555 - val_accuracy: 0.3964
Epoch 5/10
443/443 [==============================] - 16s 37ms/step - loss: 1.3011 - accuracy: 0.5925 - val_loss: 2.2343 - val_accuracy: 0.3672
Epoch 6/10
443/443 [==============================] - 16s 35ms/step - loss: 0.8163 - accuracy: 0.7394 - val_loss: 2.8462 - val_accuracy: 0.3316
Epoch 7/10
443/443 [==============================] - 16s 35ms/step - loss: 0.4908 - accuracy: 0.8466 - val_loss: 3.6915 - val_accuracy: 0.3100
Epoch 8/10
443/443 [==============================] - 16s 36ms/step - loss: 0.3246 - accuracy: 0.9027 - val_loss: 4.8499 - val_accuracy: 0.2916
Epoch 9/10
443/443 [==============================] - 16s 36ms/step - loss: 0.2025 - accuracy: 0.9392 - val_loss: 4.7970 - val_accuracy: 0.2719
Epoch 10/10
443/443 [==============================] - 16s 36ms/step - loss: 0.1621 - accuracy: 0.9524 - val_loss: 6.1159 - val_accuracy